# 문제 6

[Kaggle 형] train_prob.csv로 문제 failure 예측하는 모델을 만들고, 

test_prob.csv에 대한 failure가 1일 확률 예측하여 다음과 같은 형식의 answer6.csv를 만들어라. 

측정 지표는 AUC(area under of ROC curve)이다. id 는 테스트 케이스의 id 이고, failure에는 failure가 1이 될 확률이다.

id,failure

16115, 0.1

16116, 0.2


**강사: 멀티캠퍼스 강선구(sunku0316.kang@multicampus.com, sun9sun9@gmail.com)**

In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import sys
import xgboost as xgb

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels, xgb]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1
xgboost 0.80


In [3]:
df_train = pd.read_csv('train_prob.csv', index_col=['id'])
df_test = pd.read_csv('test_prob.csv', index_col=['id'])
df_ans = pd.read_csv('test_prob_ans.csv', index_col='id')

In [6]:
df_train = df_train.assign(
    na_1 = lambda x: x['measurement_3'].isna(),
    na_2 = lambda x: x['measurement_5'].isna()
)
df_test = df_test.assign(
    na_1 = lambda x: x['measurement_3'].isna(),
    na_2 = lambda x: x['measurement_5'].isna()
)

In [7]:
df_train['product_code'].value_counts()

C    5765
E    5343
B    5250
A    5100
Name: product_code, dtype: int64

In [8]:
df_test['product_code'].value_counts()

D    5112
Name: product_code, dtype: int64

In [11]:
# 방법 2: product_code 별로 IterativeImputer를 통한 결측처리를 하고 바로 적용합니다.
from sklearn.experimental import enable_iterative_imputer# 구문을 사용하여 실험 단계인 모듈을 활성화하고, 
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

X_imp = ['measurement_{}'.format(i) for i in range(3, 10)] + ['measurement_17']

imp = IterativeImputer(
    estimator = LinearRegression(),
    random_state=123
)

df_train[X_imp] =  df_train.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x), index=x.index, columns=x.columns)
)

df_test[X_imp] =  df_test.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x), index=x.index, columns=x.columns)
)

In [12]:
# 방법 1: groupby ~ transform
# Transform을 통해 수준별 평균으로 1:1 변환된 값으로 구성된 DataFrame을 받아서, 
# 각각의 요소별로 결측이면 해당값으로 치환되게 구성합니다.
X_mean = ['measurement_{}'.format(i) for i in range(10, 17)]
df_train[X_mean] = df_train.groupby('product_code')[X_mean]\
        .transform(lambda x: x.fillna(x.mean()))

df_test[X_mean] = df_test.groupby('product_code')[X_mean]\
        .transform(lambda x: x.fillna(x.mean()))

In [13]:
df_train['loading'] = df_train['loading'].fillna(df_train['loading'].mean())
df_test['loading'] = df_test['loading'].fillna(df_test['loading'].mean())

In [16]:
df_train.isna().sum().sum(), df_test.isna().sum().sum()

(0, 0)

# Kaggle형 풀이 단계

Step 1: 검증 방법을 정하고, 검증 루틴을 만듭니다.

Step 2: Baseline 모델을 만듭니다

Step 3: 모델 선택 루틴을 만듭니다.

Step 4: 모델 개선 작업을 합니다.

In [ ]:
# Baseline: feature selection

('loading',
  'measurement_1',
  'measurement_4',
  'measurement_14',
  'measurement_17',
  'na_1')

In [ ]:
# + np.log (loading)

In [ ]:
# + feature

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
X_lda = ['measurement_{}'.format(i) for i in range(18)]
lda.fit(df_prob3_train[X_lda], df_prob3_train['failure'])

In [ ]:
# + feature
from sklearn.decomposition import PCA
n_components=7

In [ ]:
# + model 
from sklearn.ensemble import RandomForestClassifier
{'max_depth': 7, 'min_samples_split': 512, 'n_estimators': 15}